# Estudo de Caso 3 - Identificação de Anomalias em Texto com Pytorch

O objetivo deste Jupyter Notebook é utilizar um modelo de machine learning com inteligência artificial para identificar frases que contém ou não fake news. Para isso, será utilizado liguagem Python e o Pytorch. 

## 1. Instalação dos Pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print('A versão da linguagem Python utilizada neste Jupyter Notebook é: ', python_version())

A versão da linguagem Python utilizada neste Jupyter Notebook é:  3.9.13


In [2]:
# Instala versão do Pytorch
!pip install -q torch==2.0.0

In [3]:
# Instala versão do Transformers
!pip install -q transformers==4.28.1

In [4]:
# Carga dos pacotes
import torch
import sklearn
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [5]:
# Mostra somente mensagens de erro
from transformers import logging
logging.set_verbosity_error()

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Estudo de Caso 3 - Análise de Texto" --iversions

Author: Estudo de Caso 3 - Análise de Texto

torch       : 2.0.0
transformers: 4.28.1
numpy       : 1.21.5
sklearn     : 1.0.2



## 2. Construção da Classe de Tokenização dos Dados

In [7]:
# Classe para tokenização dos dados
class TokenizaDados(Dataset):
    
    # Método construtor
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    # Método para calcular o comprimento do texto (cada sentença)
    def __len__(self):
        return len(self.texts)

    # Método para obter um item tokenizado
    def __getitem__(self, idx):
        
        # Obtém o índice do texto e do label
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Aplica a tokenização
        inputs = self.tokenizer.encode_plus(text,
                                            add_special_tokens = True,
                                            max_length = self.max_length,
                                            padding = 'max_length',
                                            truncation = True,
                                            return_tensors = 'pt')

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label)
        }

## 3. Funções para Loop de Treino e Avaliação

In [8]:
# Método do loop de treino
def treina_modelo(model, data_loader, criterion, optimizer, device):
    
    # Coloca o modelo em modo de treino
    model.train()
    
    # Inicializa o erro com zero
    total_loss = 0

    # Loop pelo data loader
    for batch in data_loader:
        
        # Extrai os ids do batch de dados e coloca no device
        input_ids = batch['input_ids'].to(device)
        
        # Extrai a máscara e coloca no device
        attention_mask = batch['attention_mask'].to(device)
        
        # Extrai os labels e coloca no device
        labels = batch['label'].to(device)

        # Zera os gradientes
        optimizer.zero_grad()
        
        # Faz as previsões
        outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
        
        # Extrai o erro do modelo
        loss = outputs.loss
        
        # Aplica a otimização com backpropagation
        loss.backward()
        optimizer.step()

        # Acumula o erro
        total_loss += loss.item()

    return total_loss / len(data_loader)

In [9]:
# Método do loop de avaliação
def avalia_modelo(model, data_loader, criterion, device):
    
    model.eval()
    
    total_loss = 0

    with torch.no_grad():
    
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            
            attention_mask = batch['attention_mask'].to(device)
            
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
            
            loss = outputs.loss
            
            total_loss += loss.item()

    return total_loss / len(data_loader)

In [10]:
# Método do loop de previsão
def predict(model, data_loader, device):
    
    model.eval()
    
    predictions = []

    with torch.no_grad():
        
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask = attention_mask)
            
            _, preds = torch.max(outputs.logits, dim = 1)
            
            predictions.extend(preds.tolist())

    return predictions

## 4. Definição dos Dados

In [11]:
# Hiperparâmetros
EPOCHS = 10
BATCH_SIZE = 16
MAX_LENGTH = 64
LEARNING_RATE = 2e-5
RANDOM_SEED = 42

In [12]:
# Conjunto de dados de exemplo
texts = [
    'A velocidade da luz é aproximadamente 300.000 km/s.',
    'A Terra é plana e os répteis controlam o mundo.',
    'A fotossíntese é um processo importante para as plantas.',
    'As vacas podem voar e atravessar paredes de concreto.',
    'O oxigênio é essencial para a respiração dos seres vivos.',
    'Os cavalos podem falar como seres humanos.',
    'As crianças aprendem a partir dos exemplos dos pais.',
    'As palavras verdadeiras não são agradáveis e as agradáveis não são verdadeiras.',
    'Leopardos trabalham de terno e gravata em frente ao computador.',
    'Carros voadores estão por toda parte.'
]

In [13]:
labels = [0, 1, 0, 1, 0, 1, 0, 0, 1, 1]  # 0: normal, 1: anômala

In [14]:
# Divisão dos dados em treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, 
                                                                      labels, 
                                                                      test_size = 0.2, 
                                                                      random_state = RANDOM_SEED)

## 5. Tokenização dos Dados e Criação dos DataLoaders

In [15]:
# Nome do modelo pré-treinado com 110M de parâmetros
PRETRAINED_MODEL = 'bert-base-uncased'

Maiores informações do modelo podem ser acessadas em:
https://huggingface.co/bert-base-uncased

In [16]:
# Inicializa o tokenizador
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL)

In [17]:
# Tokenização dos dados
train_dataset = TokenizaDados(train_texts, train_labels, tokenizer, MAX_LENGTH)
test_dataset = TokenizaDados(test_texts, test_labels, tokenizer, MAX_LENGTH) 

In [18]:
# Data Loaders
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [19]:
# Configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 6. Construção, Treinamento e Avaliação do Modelo

In [20]:
# Importa o modelo pré-treinado
modelo = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels = 2)

In [21]:
# Coloca o modelo na memória do device
modelo.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [22]:
# Configuração do otimizador e critério de perda
optimizer = torch.optim.AdamW(modelo.parameters(), lr = LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [23]:
# Treinamento e Avaliação do Modelo
for epoch in range(EPOCHS):
    
    train_loss = treina_modelo(modelo, train_loader, criterion, optimizer, device)
    
    test_loss = avalia_modelo(modelo, test_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Test Loss: {test_loss}')

Epoch 1/10, Train Loss: 0.6740316152572632, Test Loss: 1.1876635551452637
Epoch 2/10, Train Loss: 0.6730251312255859, Test Loss: 1.0781298875808716
Epoch 3/10, Train Loss: 0.5299704074859619, Test Loss: 1.0337677001953125
Epoch 4/10, Train Loss: 0.33613672852516174, Test Loss: 1.1865148544311523
Epoch 5/10, Train Loss: 0.2950790524482727, Test Loss: 0.9976953864097595
Epoch 6/10, Train Loss: 0.3000965416431427, Test Loss: 0.5799145698547363
Epoch 7/10, Train Loss: 0.23467014729976654, Test Loss: 0.26405900716781616
Epoch 8/10, Train Loss: 0.22346428036689758, Test Loss: 0.2626678943634033
Epoch 9/10, Train Loss: 0.2167423516511917, Test Loss: 0.2882377803325653
Epoch 10/10, Train Loss: 0.20056995749473572, Test Loss: 0.4970722794532776


In [24]:
## 7. Deploy e Uso do Modelo Treinado


In [25]:
# Teste de detecção de anomalias
novos_dados = ['A gravidade mantém os planetas em órbita ao redor do Sol.',
               'Os carros podem nadar no oceano como peixes.']

In [26]:
# Tokeniza a amostra de dados
novo_dataset = TokenizaDados(novos_dados, [0] * len(novos_dados), tokenizer, MAX_LENGTH)

In [27]:
# Cria o dataloader
novo_loader = DataLoader(novo_dataset, batch_size = BATCH_SIZE)

In [28]:
# Faz as previsões com o modelo
previsoes = predict(modelo, novo_loader, device)

In [29]:
for text, prediction in zip(novos_dados, previsoes):
    print(f'Sentença: {text} | Previsão: {"anômala" if prediction else "normal"}')

Sentença: A gravidade mantém os planetas em órbita ao redor do Sol. | Previsão: normal
Sentença: Os carros podem nadar no oceano como peixes. | Previsão: anômala
